Задача состояла в предсказании продаж для сдвига на 1,2,3 недели от предоставленных данных по аналитике продаж

Посмотрев на начальные данные я заметила что данные для некоторых недель с различным смещением совпадают, для тех строк, у которых значение (недели - смещение)  совпадало.


Предположение 1: если поставить для каждой строки в соответствие значения этой недели(то есть рассмотреть неделю с номером (недели - смещение)  для целевой) то предсказания будут более точными(код и заполнение в ноутбуке с Baseline)

Предположение 2: нужно воспользоваться gridSearch для подбора параметров модели, рассмотреть несколько моделей(кросс-вализация для SMAPE)

Предположение 3: то же, что 1, но смещение в одну неделю. Потому что не для всех строк можно подобрать строчку с данными без смещения. Обучить регрессию для единичного смещения, для всех данных, где нельзя подобрать данные соответствующие данной неделе, выдать результат на этой модели.(Этот вариант дал результат не лучше чем просто пременение 1)

Оценивалось решение по SMAPE на кросс-валидации на train.

In [ ]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import numpy as np

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
test_new = pd.read_csv("test_new.tsv")
train_new = pd.read_csv("train_new.tsv")

In [2]:
test.head()

,Num,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,348622,2015,3,3,20447918,960.0,820.0,1128.0,1801.0,1045.0,...,1510.0,580.0,969.0,1635.0,895.0,2140.0,1182.0,1020.0,1293.0,1290.0
1,348623,2015,3,3,20447902,9086.0,12585.0,11595.0,9685.0,12917.0,...,22055.0,14235.0,21195.0,18280.0,18270.0,15851.0,16920.0,18320.0,24116.0,21307.0
2,348624,2015,3,3,20447732,115087.0,147287.0,176065.0,143105.0,202069.0,...,302165.0,162232.0,221622.0,256605.0,240047.0,236630.0,206697.0,245652.0,286179.0,285904.0
3,348625,2015,3,3,20443951,20900.0,24420.0,27068.0,20460.0,25580.0,...,39055.0,14445.0,22450.0,22093.0,31175.0,23355.0,15358.0,18930.0,29643.0,33970.0
4,348626,2015,3,3,20443944,4430.0,5864.0,3310.0,1853.0,2836.0,...,120.0,130.0,60.0,30.0,50.0,20.0,20.0,30.0,0.0,0.0


In [3]:
test_new.head()

,Unnamed: 0,Num,year,week,shift,item_id,f1,f2,f3,f4,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,0,348622,2015,3,0,20447918,1801.0,1045.0,1190.0,1310.0,...,1635.0,895.0,2140.0,1182.0,1020.0,1293.0,1290.0,422.0,1690.0,930.0
1,1,348623,2015,3,0,20447902,9685.0,12917.0,11157.0,12105.0,...,18280.0,18270.0,15851.0,16920.0,18320.0,24116.0,21307.0,6661.0,20610.0,16791.0
2,2,348624,2015,3,0,20447732,143105.0,202069.0,216920.0,236053.0,...,256605.0,240047.0,236630.0,206697.0,245652.0,286179.0,285904.0,54846.0,171320.0,184820.0
3,3,348625,2015,3,0,20443951,20460.0,25580.0,27135.0,24059.0,...,22093.0,31175.0,23355.0,15358.0,18930.0,29643.0,33970.0,7787.0,20395.0,18305.0
4,4,348626,2015,3,0,20443944,1853.0,2836.0,2807.0,2550.0,...,30.0,50.0,20.0,20.0,30.0,0.0,0.0,0.0,10.0,10.0


In [79]:
frac = 0.1 # fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
train_new = train_new.sample(frac=frac, random_state=42)

X = train_new.drop(['Num','y'], axis=1)
y = train_new['y']


In [14]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=43)
model.fit(X, y)

preds = model.predict(test_new.drop(['Num'], axis=1))

print len(preds)
print len(sample_submission)

2016
2016
CPU times: user 59.4 s, sys: 32 ms, total: 59.4 s
Wall time: 59.4 s


In [15]:
sample_submission = pd.read_csv("sample_submission.tsv")
sample_submission['y'] = preds
sample_submission.head(5)

,Num,y
0,348622,14910.434067
1,348623,47591.392085
2,348624,333537.167465
3,348625,49336.563421
4,348626,13069.743997


In [16]:
print sample_submission[sample_submission['y'] < 0]

Empty DataFrame
Columns: [Num, y]
Index: []


In [17]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [18]:
sample_submission.to_csv("my_submission_1.tsv", sep=',', index=False)

In [19]:
def smape(y, f, **kwargs):
    res = 2. * 100. / float(len(y))
    y = list(map(float, y))
    f = list(map(float, f))
    res *= np.sum([abs(f[i] - y[i]) / (abs(y[i]) + abs(f[i])) for i in range(len(y))])
    return - res

smape_score = metrics.make_scorer(smape)

In [45]:
%%time
model = GradientBoostingRegressor(n_estimators=200, max_depth=7,  random_state=43)
print(np.average(cross_val_score(model, X, y, scoring=smape_score)))

-12.8483327463
CPU times: user 8min 36s, sys: 160 ms, total: 8min 36s
Wall time: 8min 36s


In [21]:
from sklearn.model_selection import GridSearchCV
parameters = {"n_estimators": np.arange(100, 500, 100), 'max_depth': np.arange(1, 20, 3),
             "random_state":[42]}
boost =  GradientBoostingRegressor()
regress = GridSearchCV(boost, parameters, scoring=smape_score)
regress.fit(X, y)
print(regress.best_estimator_, regress.best_score_)


KeyboardInterrupt: 

In [82]:
%%time
train_new = pd.read_csv("train_new.tsv")
X = train_new.drop(['Num','y'], axis=1)
y = train_new['y']

model = GradientBoostingRegressor(n_estimators=200, max_depth=7,  random_state=43)
print(np.average(cross_val_score(model, X, y, scoring=smape_score)))

-12.8483327463
CPU times: user 8min 41s, sys: 248 ms, total: 8min 42s
Wall time: 8min 42s


In [81]:
%%time
train_new = pd.read_csv("train_new.tsv")
X = train_new.drop(['Num','y'], axis=1)
y = train_new['y']

model_four= GradientBoostingRegressor(n_estimators=200, max_depth=5,  random_state=43)
print(np.average(cross_val_score(model_four, X, y, scoring=smape_score)))

KeyboardInterrupt: 

In [80]:
X

,Unnamed: 0,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
44283,44283,2014,13,0,20452327,104610.0,121114.0,133780.0,122580.0,126830.0,...,88520.0,107270.0,79610.0,99210.0,114561.0,93790.0,98070.0,83980.0,105240.0,99690.0
50871,50871,2014,23,0,20441989,6760.0,7210.0,11330.0,6950.0,6798.0,...,8908.0,10464.0,7431.0,10334.0,11548.0,8698.0,8696.0,8160.0,7570.0,8141.0
13810,13810,2013,21,0,20438706,37930.0,21420.0,28240.0,28685.0,39205.0,...,29660.0,30105.0,28054.0,31545.0,28185.0,34890.0,28790.0,25630.0,32575.0,33169.0
10062,10062,2013,15,0,20438591,15360.0,14750.0,12961.0,9880.0,11950.0,...,8060.0,8430.0,9980.0,13930.0,6340.0,7810.0,8960.0,10260.0,9180.0,6010.0
37186,37186,2014,3,0,20449525,0.0,0.0,0.0,0.0,0.0,...,1145.0,2955.0,3915.0,740.0,2260.0,1403.0,1417.0,980.0,255.0,830.0
11779,11779,2013,17,0,20442117,30476.0,27444.0,35176.0,37784.0,61653.0,...,22744.0,47030.0,17352.0,18392.0,20060.0,35480.0,18256.0,16277.0,14780.0,17833.0
19487,19487,2013,28,0,20427463,8900.0,14780.0,5194.0,4070.0,8050.0,...,7600.0,11130.0,11349.0,16290.0,7410.0,5360.0,10620.0,12701.0,8141.0,8910.0
13814,13814,2013,21,0,20438803,134052.0,61791.0,56961.0,80549.0,96744.0,...,81732.0,140618.0,142027.0,67727.0,74731.0,102966.0,117584.0,81805.0,123387.0,125871.0
9022,9022,2013,13,0,20442103,816429.0,825582.0,1022983.0,1584683.0,1117102.0,...,949393.0,852544.0,730940.0,807951.0,894810.0,1263319.0,582537.0,742668.0,756275.0,775553.0
47174,47174,2014,18,0,20448095,52926.0,55286.0,50816.0,64685.0,27994.0,...,62497.0,57887.0,48204.0,72346.0,64235.0,69257.0,67125.0,68337.0,76017.0,80863.0


In [83]:
model.fit(X, y)
preds = model.predict(test_new.drop(['Num'], axis=1))

In [47]:
test_new_1 = pd.read_csv("test_for_1.tsv")
train_new_1 = pd.read_csv("train_for_1.tsv")

In [48]:
X_1 = train_new_1.drop(['Num','y'], axis=1)
y_1 = train_new_1['y']

model_1 = GradientBoostingRegressor(n_estimators=200, max_depth=7,  random_state=43)
print(np.average(cross_val_score(model_1, X_1, y_1, scoring=smape_score)))

-36.0962571516


In [69]:
preds_1

array([   8488.72700627,   29824.20738166,  280548.30705655, ...,
         19913.17378574,     697.94647899,    6433.38692665])

In [65]:
model_1.fit(X_1, y_1)
preds_1 = model_1.predict(test_new_1.drop(['Num'], axis=1))


In [74]:
test[1900:]

,Num,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
1900,351413,2015,5,1,20450610,657280.0,691028.0,684270.0,833466.0,527433.0,...,907583.0,826757.0,626970.0,760267.0,1104606.0,1212613.0,175709.0,666235.0,787459.0,969078.0
1901,351414,2015,5,1,20450611,222463.0,215313.0,229167.0,264106.0,223816.0,...,259790.0,228939.0,174453.0,212106.0,319582.0,322878.0,51374.0,165716.0,208760.0,277809.0
1902,351415,2015,5,1,20450612,0.0,5890.0,3745.0,7635.0,56025.0,...,24370.0,19820.0,19964.0,26680.0,27165.0,31229.0,5614.0,19970.0,19531.0,36373.0
1903,351416,2015,5,1,20450671,134435.0,133828.0,143425.0,147861.0,135733.0,...,122998.0,111326.0,112468.0,132030.0,170638.0,156225.0,31880.0,123445.0,103470.0,125116.0
1904,351417,2015,5,1,20450803,0.0,0.0,0.0,0.0,0.0,...,575.0,230.0,683.0,850.0,1083.0,485.0,164.0,450.0,520.0,1041.0
1905,351418,2015,5,1,20450856,137540.0,162548.0,170560.0,200590.0,120215.0,...,134261.0,131887.0,93902.0,114820.0,201424.0,164206.0,24647.0,108026.0,120701.0,150266.0
1906,351419,2015,5,1,20438605,2212821.0,2169261.0,2230321.0,2803885.0,1806521.0,...,2090835.0,1772850.0,1650111.0,1798731.0,2648303.0,2453944.0,422352.0,1369720.0,1547868.0,1941151.0
1907,351420,2015,5,1,20438537,30330.0,35080.0,45255.0,111365.0,28820.0,...,35972.0,30165.0,19370.0,24022.0,42165.0,55146.0,4680.0,16530.0,13585.0,43975.0
1908,351421,2015,5,1,20438538,662205.0,736559.0,762739.0,908625.0,605715.0,...,643429.0,472584.0,425782.0,539221.0,812283.0,773216.0,123650.0,371323.0,439735.0,574232.0
1909,351422,2015,5,1,20438539,540235.0,573930.0,583420.0,723860.0,487581.0,...,545277.0,406932.0,335378.0,417522.0,640036.0,636057.0,82952.0,314626.0,339787.0,424382.0


In [87]:
best_preds = pd.read_csv("my_submission_2.tsv")['y']

ids = test_new.index
for i in ids:
    if test_new.loc[i, 'shift'] !=0:
        print("id %d: %d-> %d" % (i, best_preds[i], preds_1[i]))
        best_preds[i] = preds_1[i]


id 455: 113-> 2104
id 607: 91-> 291
id 608: 288-> 264
id 625: 125-> 265
id 626: 125-> 6999
id 653: 707-> 233
id 654: 34-> 233
id 907: 7-> 2108
id 1059: 13-> 295
id 1060: 0-> 268
id 1077: 19-> 269
id 1078: 19-> 7004
id 1105: 0-> 249
id 1106: 0-> 249
id 1130: 233119-> 118613
id 1131: 81378-> 56733
id 1132: 2585-> 3479
id 1133: 3017-> 3479
id 1134: 38671-> 28641
id 1135: 439330-> 224814
id 1136: 57272-> 33996
id 1137: 189412-> 112219
id 1138: 123972-> 72504
id 1139: 121396-> 101319
id 1140: 124822-> 87309
id 1141: 3719-> 3021
id 1142: 2063-> 2182
id 1143: 195613-> 89796
id 1144: 165787-> 83099
id 1145: 531450-> 294044
id 1146: 542658-> 290541
id 1147: 2004-> 3704
id 1148: 166625-> 104563
id 1149: 124716-> 85286
id 1150: 1627-> 2868
id 1151: 25786-> 33119
id 1152: 11087-> 12205
id 1153: 197628-> 110551
id 1154: 11027-> 12569
id 1155: 27762-> 43453
id 1156: 104424-> 63608
id 1157: 502165-> 209609
id 1158: 368315-> 148561
id 1159: 16773-> 8298
id 1160: 31516-> 27913
id 1161: 17890-> 12753
id

In [88]:
sample_submission = pd.read_csv("sample_submission.tsv")
sample_submission['y'] = best_preds

print sample_submission[sample_submission['y'] < 0]
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

sample_submission.to_csv("my_submission_3.tsv", sep=',', index=False)

         Num           y
1176  350689  -80.766175
1320  350833 -353.468687
1622  351135  -80.766175
1766  351279 -353.468687
